In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
from keras.models import Model, load_model
import glob 
import  pandas as pd
import matplotlib.pyplot as plt
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import os
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
### set parameters
data_path= 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/data/'
preprocessor_dir='G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/preprocessor/'
preprocessor_name='preprocessor.p'
model_dir='G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/model/'
model_name= 'model.h5'
selected_Columns= ['t_spindle', 't_motor','supply_temp']
window= 10

In [ ]:
def generateDataSource(signal_input=None, input_columns: list = [], output_length: int = 1, signal_output=None, window=1, shift=1):
    Signal_Length = signal_input.shape[0]
    num_samples = int((Signal_Length - window + 1) / shift)
    x = np.zeros(shape=(num_samples, window, signal_input.shape[1]))
    y = np.zeros(shape=(num_samples, output_length, 1))
    for i in range(num_samples):
        x[i] = signal_input[i * shift: i * shift + window]
        y[i] = signal_output[i * shift + window -
                             1: i * shift + window - 1 + output_length]
    return x, y

In [ ]:
# Implement special preprocessing steps
# 1 calculate new Feature
# 2 make rounding


from sklearn.preprocessing import FunctionTransformer


def special_preprocess(data: np.ndarray):
    delta_t_spindel_bett = data[:, 1] - data[:, 0]
    delta_t_spindel_bett = delta_t_spindel_bett.reshape((-1, 1))
    result = np.concatenate((data[:, 2:5], delta_t_spindel_bett), axis=1)
    return result


def rounding(signals: np.ndarray):
    return np.round(signals, 2)


def remove_t_bett(data: np.ndarray):
    result = data[:, 1:]
    result[:, 0] = result[:, 0] - data[:, 0]
    result[:, 1] = result[:, 1] - data[:, 0]
    return result

In [ ]:
### load models
pipeline= load(str(preprocessor_dir+preprocessor_name))
print('preprocessor is loaded successfully')
model= load_model(str(model_dir + model_name), compile= False)
print('Model is loaded successfully')

In [ ]:
def filter_centerfugal_force_effect(df: pd.DataFrame):
    factor= 6.58*(10**(-7))
    result=  factor * df['DRZ'].abs()#np.round(df['DRZ'].abs(),-3)
    return result


In [ ]:
selected_Columns= ['t_bett','t_motor','t_spindle','given2model']#'date','t_bett','t_motor','t_spindle','given2model','welle_z','used4spline','spline_interp'

In [ ]:
### Load Data
data_path= 'G:\\Innovations@HELLER\\DN\\KI\\AutoLern\\Temperaturkompensation_Datensaetze\\'
scatter_mode= 'lines'
training_columns= ['t_bett','t_motor', 't_spindle','M8', 'M121', 'M127', 'M7']
os.chdir(data_path)
for file in glob.glob('*.csv'):
    df= pd.read_csv(file)
    training_df= df[selected_Columns]
    training_df['M8']=0
    training_df['M121']=0
    training_df['M127']=0
    training_df['M7']=0
    signals= training_df[training_columns]
    signals= signals.to_numpy()
    input= pipeline.transform(signals)
    output= df['welle_z']
    partitions, target= generateDataSource(signal_input= input, input_columns= selected_Columns, output_length= 1,signal_output= output, window= window, shift= 1)
    target= np.reshape(target,-1)
    predictions= model.predict(partitions)
    predictions= np.reshape(predictions,-1)
    print(predictions.shape)
    print (' mae= ', np.mean(mae(target,predictions)))
    ######
    visualization_columns= ['t_bett','t_motor','t_spindle','welle_z','spline_interp']
    df= df[window-1:]
    print(df.shape)
    df['welle_z']= df['welle_z'].shift(-4)
    print(df.shape)
    y_axis_names= visualization_columns# ['ENC1_POS|8 (degee)     ', 'CMD_SPEED|8 (degree/s)    ','is_speed (degree/s)   ', 'is_acceleration(degree/s^2)   ',  'POWER|8 (W)',  'CURRENT|8 (A)',  'ENC2_POS|3 (mm)','t_spindle', 'DRZ', 't_motor','supply_temp', 't_bett', 'distance']
    fig= make_subplots(rows=len(visualization_columns)-1,cols=1,shared_xaxes= True, print_grid= True, subplot_titles= visualization_columns, vertical_spacing=0.02)
    for i in range(len(visualization_columns)):
        if i == len(visualization_columns) -1:
            fig.add_trace(go.Scatter(x= df['date'],y= df[visualization_columns[i]], name=visualization_columns[i], mode= scatter_mode), row= i, col= 1)
            fig.update_yaxes(title_text= y_axis_names[i], row= i, col= 1)
        else:
            fig.add_trace(go.Scatter(x= df['date'],y= df[visualization_columns[i]], name=visualization_columns[i], mode= scatter_mode), row= i+1, col= 1)
            fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
    fig.add_trace(go.Scatter(x= df['date'],y= predictions / 1000, name='Model prediction', mode= scatter_mode), row= len(visualization_columns) -1, col= 1) 
    fig.update_layout(height=1800, width=1200, title_text="5 Sekunden-Daten")
    fig.show()
    

In [ ]:
### Load Data
os.chdir(data_path)
for file in glob.glob('*.csv'):
    df= pd.read_csv(file)
    
    input_df= df['distance'].copy()
    input_df= input_df.shift(periods= -1, axis= 0, fill_value= 0)
    df['distance']= input_df
    df= df.iloc[:2000]
    print('current File: ', file)
    #distance= df['distance']
    df['correction']= filter_centerfugal_force_effect(df)
    df['new_distance']= df['distance'] + df['correction']# df['distance']#
    df['displacement']= df['new_distance'][0] - df['new_distance']

    selected_columns= ['t_spindle', 't_motor','t_welle', 'DRZ', 'distance','correction','new_distance','displacement']
    y_axis_names= selected_columns# ['ENC1_POS|8 (degee)     ', 'CMD_SPEED|8 (degree/s)    ','is_speed (degree/s)   ', 'is_acceleration(degree/s^2)   ',  'POWER|8 (W)',  'CURRENT|8 (A)',  'ENC2_POS|3 (mm)','t_spindle', 'DRZ', 't_motor','supply_temp', 't_bett', 'distance']
    fig= make_subplots(rows=len(selected_columns),cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_columns, vertical_spacing=0.02)
    for i in range(len(selected_columns)):
        fig.add_trace(go.Line(y= df[selected_columns[i]]), row= i+1, col= 1)
        fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
    fig.update_layout(height=1800, width=1200, title_text="LF-Data")
    fig.show()

    signals= df[selected_Columns]
    signals= signals.to_numpy()
    input= pipeline.transform(signals)
    output= df['displacement']
    partitions, target= generateDataSource(signal_input= input, input_columns= selected_Columns, output_length= 1,signal_output= output, window= window, shift= 1)
    target= np.reshape(target,-1)
    predictions= model.predict(partitions)
    predictions= np.reshape(predictions,-1)
    print (' mae= ', np.mean(mae(target,predictions)))
    # fig, ax= plt.subplots(figsize=(15,15))
    # plt.title(str('Results of LSTM Algorithm on File '+ file))
    # plt.xlabel('N')
    # plt.ylabel('Kompensation (mm)')
    # plt.grid()
    # plt.plot(target, label= 'Orginal z_tcp_ok (Soll_Werte)', color='green')
    # plt.plot(predictions, label= 'z_tcp_ok (Prognose)', color='red')
    # #plt.plot(komp, label= 'kompensation (Ist_Werte)', color='blue')
    # plt.legend()
    # plt.show()
    ## Visualization of lf-data with predicted results
    result_df= pd.DataFrame({'displacement': target, 'prediction': predictions})
    fig= ex.line(result_df,y=['displacement', 'prediction'])
    #fig.update_layout(height=1200, width=1200, title_text="Laser-Data")
    fig.show()
        

In [ ]:
    fig= ex.line(y=abs(result_df['displacement'] - result_df['prediction']))
    #fig.update_layout(height=1200, width=1200, title_text="Laser-Data")
    fig.show()
        

In [ ]:
### Calculate predictions

In [ ]:
### visualize Results